<a href="https://colab.research.google.com/github/skywalker0803r/deep-learning-ian-goodfellow/blob/master/ADDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
from torch import nn
from torch.autograd import Function
from torch import optim
from torch.utils.data import TensorDataset,DataLoader

In [33]:
class ADDA(nn.Module):
  def __init__(self,input_dim,out_dim):
    super().__init__()
    
    # sub model 1
    self.SRC_F = nn.Sequential(
        nn.Linear(input_dim,128),
        nn.ReLU(),
        nn.Linear(128,128),
        nn.ReLU(),
        )
    # sub model 2
    self.TGT_F = nn.Sequential(
        nn.Linear(input_dim,128),
        nn.ReLU(),
        nn.Linear(128,128),
        nn.ReLU(),
        )
    # sub model 3
    self.regression = nn.Sequential(
        nn.Linear(128,128),
        nn.ReLU(),
        nn.Linear(128,out_dim),
        )
    # sub model 4
    self.discriminator = nn.Sequential(
        nn.Linear(128,128),
        nn.ReLU(),
        nn.Linear(128,1),
        nn.Sigmoid(),
    )
    
    # optimizer
    self.S_optimizer = optim.Adam(list(self.SRC_F.parameters()))
    self.R_optimizer = optim.Adam(list(self.regression.parameters()))
    self.T_optimizer = optim.Adam(list(self.TGT_F.parameters()))
    self.D_optimizer = optim.Adam(list(self.discriminator.parameters()))
    
    # loss_function
    self.regression_loss = nn.MSELoss()
    self.discriminator_loss = nn.BCELoss()
    
  
  def forward(self):
    return NotImplementedError
  
  def train_stage1(self,x,y):
    x = self.SRC_F(x)
    y_hat = self.regression(x)
    
    loss = self.regression_loss(y_hat,y)
    loss.backward()
    
    self.S_optimizer.step()
    self.R_optimizer.step()
    
    self.S_optimizer.zero_grad()
    self.R_optimizer.zero_grad()
    return loss.item()
  
  def train_stage2(self):
    raise NotImplementedError

In [31]:
model = ADDA(4,23)

In [32]:
x,y = torch.rand(1000,4),torch.rand(1000,23)
data_iter = DataLoader(TensorDataset(x,y))
for i in range(100):
  for x,y in data_iter:
    loss = model.train_stage1(x,y)
  print("ep:{} loss:{}".format(i,loss))

ep:0 loss:0.08256270736455917
ep:1 loss:0.08274053782224655
ep:2 loss:0.08229213953018188
ep:3 loss:0.08158183842897415
ep:4 loss:0.0812436044216156
ep:5 loss:0.08062288910150528
ep:6 loss:0.08087152242660522
ep:7 loss:0.08133424073457718
ep:8 loss:0.0797230452299118
ep:9 loss:0.07927369326353073
ep:10 loss:0.07942277193069458
ep:11 loss:0.07864271104335785
ep:12 loss:0.08038663864135742
ep:13 loss:0.0812470093369484
ep:14 loss:0.08155708014965057
ep:15 loss:0.08217764645814896
ep:16 loss:0.08268892019987106
ep:17 loss:0.08249996602535248
ep:18 loss:0.0821952074766159
ep:19 loss:0.07804698497056961
ep:20 loss:0.07801053673028946
ep:21 loss:0.08231271058320999
ep:22 loss:0.07940320670604706
ep:23 loss:0.07880418002605438
ep:24 loss:0.07555436342954636
ep:25 loss:0.07446710765361786
ep:26 loss:0.074522003531456
ep:27 loss:0.07367048412561417
ep:28 loss:0.07647020369768143
ep:29 loss:0.07614503800868988
ep:30 loss:0.07427051663398743
ep:31 loss:0.07366521656513214
ep:32 loss:0.07529757171

KeyboardInterrupt: ignored